import libraries

In [ ]:
import json
import numpy as np
import pandas as pd
import numpy as np
import xgboost as xgb
import matplotlib.pyplot as plt


those are functions to combine and seperate the test and train dataset

In [70]:
def combine_datasets(train_data, test_data):
    # Combine the training and test datasets
    combined_df = pd.concat([train_data, test_data], ignore_index=True)

    return combined_df

def separate_datasets(combined_df, train_rows):
    # Separate the combined DataFrame into training and test datasets
    train_combined = combined_df.iloc[:train_rows, :]
    test_combined = combined_df.iloc[train_rows:, :]

    return train_combined, test_combined

# 1.   data collection

In [71]:
import pandas as pd

# Spécifiez le chemin vers votre fichier JSON

# Chargez les données JSON dans un DataFrame pandas
df1 = pd.read_json('data/train/underwritings.json')

# Écrivez le DataFrame dans un fichier CSV
df1.to_csv("data/train/underwritings.csv", index=False)

df1=pd.read_csv("data/train/underwritings.csv")

In [72]:
df_test=pd.read_csv("data/submission/test.csv")

In [73]:
df1.shape, df_test.shape

((48752, 54), (1000, 53))

In [74]:
df2 = df1.drop(columns='clearfraudscore')

In [75]:
df2.shape, df_test.shape

((48752, 53), (1000, 53))

In [76]:
df = combine_datasets(df2, df_test)
df.shape

(49752, 53)

In [77]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49752 entries, 0 to 49751
Data columns (total 53 columns):
 #   Column                                                                                               Non-Null Count  Dtype  
---  ------                                                                                               --------------  -----  
 0   .underwritingdataclarity.clearfraud.clearfraudinquiry.thirtydaysago                                  49750 non-null  float64
 1   .underwritingdataclarity.clearfraud.clearfraudinquiry.twentyfourhoursago                             49750 non-null  float64
 2   .underwritingdataclarity.clearfraud.clearfraudinquiry.oneminuteago                                   49750 non-null  float64
 3   .underwritingdataclarity.clearfraud.clearfraudinquiry.onehourago                                     49750 non-null  float64
 4   .underwritingdataclarity.clearfraud.clearfraudinquiry.ninetydaysago                                  49750

# 2.   Analyse Data

In [78]:
df.isna().sum()

.underwritingdataclarity.clearfraud.clearfraudinquiry.thirtydaysago                                        2
.underwritingdataclarity.clearfraud.clearfraudinquiry.twentyfourhoursago                                   2
.underwritingdataclarity.clearfraud.clearfraudinquiry.oneminuteago                                         2
.underwritingdataclarity.clearfraud.clearfraudinquiry.onehourago                                           2
.underwritingdataclarity.clearfraud.clearfraudinquiry.ninetydaysago                                        2
.underwritingdataclarity.clearfraud.clearfraudinquiry.sevendaysago                                         2
.underwritingdataclarity.clearfraud.clearfraudinquiry.tenminutesago                                        2
.underwritingdataclarity.clearfraud.clearfraudinquiry.fifteendaysago                                       2
.underwritingdataclarity.clearfraud.clearfraudinquiry.threesixtyfivedaysago                                2
.underwritingdatacl

In [79]:
columns_to_drop = [
    '.underwritingdataclarity.clearfraud.clearfraudidentityverification.nameaddressreasoncode',
    '.underwritingdataclarity.clearfraud.clearfraudidentityverification.ssnnamereasoncode',
    '.underwritingdataclarity.clearfraud.clearfraudidentityverification.ssndobreasoncode',
    '.underwritingdataclarity.clearfraud.clearfraudidentityverification.phonetype',
    '.underwritingdataclarity.clearfraud.clearfraudidentityverification.nameaddressreasoncodedescription',
    '.underwritingdataclarity.clearfraud.clearfraudidentityverification.ssnnamereasoncodedescription',
    '.underwritingdataclarity.clearfraud.clearfraudindicator.driverlicenseinconsistentwithonfile',
    '.underwritingdataclarity.clearfraud.clearfraudindicator.workphonepreviouslylistedascellphone',
    '.underwritingdataclarity.clearfraud.clearfraudindicator.workphonepreviouslylistedashomephone',
    'underwritingid'
]

# Supprimer les colonnes spécifiées
df = df.drop(columns=columns_to_drop, axis=1)

In [80]:
df.isna().sum()

.underwritingdataclarity.clearfraud.clearfraudinquiry.thirtydaysago                                     2
.underwritingdataclarity.clearfraud.clearfraudinquiry.twentyfourhoursago                                2
.underwritingdataclarity.clearfraud.clearfraudinquiry.oneminuteago                                      2
.underwritingdataclarity.clearfraud.clearfraudinquiry.onehourago                                        2
.underwritingdataclarity.clearfraud.clearfraudinquiry.ninetydaysago                                     2
.underwritingdataclarity.clearfraud.clearfraudinquiry.sevendaysago                                      2
.underwritingdataclarity.clearfraud.clearfraudinquiry.tenminutesago                                     2
.underwritingdataclarity.clearfraud.clearfraudinquiry.fifteendaysago                                    2
.underwritingdataclarity.clearfraud.clearfraudinquiry.threesixtyfivedaysago                             2
.underwritingdataclarity.clearfraud.clearfraud

In [81]:
# Liste des colonnes à encoder en one-hot
columns_to_encode = [
    ".underwritingdataclarity.clearfraud.clearfraudidentityverification.ssnnamematch",
    ".underwritingdataclarity.clearfraud.clearfraudidentityverification.phonematchtype",
    ".underwritingdataclarity.clearfraud.clearfraudidentityverification.phonematchresult",
    ".underwritingdataclarity.clearfraud.clearfraudidentityverification.phonematchtypedescription",
    ".underwritingdataclarity.clearfraud.clearfraudidentityverification.overallmatchresult",
    ".underwritingdataclarity.clearfraud.clearfraudidentityverification.ssndobmatch",
    ".underwritingdataclarity.clearfraud.clearfraudidentityverification.nameaddressmatch"
]

# Appliquer one-hot encoding sur les colonnes spécifiées
df = pd.get_dummies(df, columns=columns_to_encode)

# Afficher le DataFrame avec les nouvelles colonnes après l'encodage


In [82]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49752 entries, 0 to 49751
Data columns (total 76 columns):
 #   Column                                                                                                                   Non-Null Count  Dtype  
---  ------                                                                                                                   --------------  -----  
 0   .underwritingdataclarity.clearfraud.clearfraudinquiry.thirtydaysago                                                      49750 non-null  float64
 1   .underwritingdataclarity.clearfraud.clearfraudinquiry.twentyfourhoursago                                                 49750 non-null  float64
 2   .underwritingdataclarity.clearfraud.clearfraudinquiry.oneminuteago                                                       49750 non-null  float64
 3   .underwritingdataclarity.clearfraud.clearfraudinquiry.onehourago                                                         49750 non-null 

In [83]:
# Convertir les données uint8 en float64
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49752 entries, 0 to 49751
Data columns (total 76 columns):
 #   Column                                                                                                                   Non-Null Count  Dtype  
---  ------                                                                                                                   --------------  -----  
 0   .underwritingdataclarity.clearfraud.clearfraudinquiry.thirtydaysago                                                      49750 non-null  float64
 1   .underwritingdataclarity.clearfraud.clearfraudinquiry.twentyfourhoursago                                                 49750 non-null  float64
 2   .underwritingdataclarity.clearfraud.clearfraudinquiry.oneminuteago                                                       49750 non-null  float64
 3   .underwritingdataclarity.clearfraud.clearfraudinquiry.onehourago                                                         49750 non-null 

In [84]:
from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=5)  
df_imputed = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)

In [85]:
df_imputed.isna().sum()

.underwritingdataclarity.clearfraud.clearfraudinquiry.thirtydaysago                                0
.underwritingdataclarity.clearfraud.clearfraudinquiry.twentyfourhoursago                           0
.underwritingdataclarity.clearfraud.clearfraudinquiry.oneminuteago                                 0
.underwritingdataclarity.clearfraud.clearfraudinquiry.onehourago                                   0
.underwritingdataclarity.clearfraud.clearfraudinquiry.ninetydaysago                                0
                                                                                                  ..
.underwritingdataclarity.clearfraud.clearfraudidentityverification.nameaddressmatch_invalid        0
.underwritingdataclarity.clearfraud.clearfraudidentityverification.nameaddressmatch_match          0
.underwritingdataclarity.clearfraud.clearfraudidentityverification.nameaddressmatch_mismatch       0
.underwritingdataclarity.clearfraud.clearfraudidentityverification.nameaddressmatch_partial

In [86]:
# Count the number of null values in each column
null_counts = df_imputed.isna().sum()

# Count the number of columns without any null values
non_null_columns_count = (null_counts == 0).sum()

print("Number of columns without any null values:", non_null_columns_count)


Number of columns without any null values: 76


# 3.   Data Preprocessing

In [87]:
train_combined, test_combined = separate_datasets(df, len(df1))

In [88]:
train_combined.shape, test_combined.shape

((48752, 76), (1000, 76))

In [89]:
from sklearn.model_selection import train_test_split

# Split the data into features (X) and target variable (y)
X = train_combined
y = df1['clearfraudscore']

In [90]:
y

0        871.0
1        397.0
2        572.0
3        838.0
4        768.0
         ...  
48747    677.0
48748    640.0
48749    576.0
48750    830.0
48751    793.0
Name: clearfraudscore, Length: 48752, dtype: float64

In [91]:
median_score = y.median()
y = y.fillna(median_score)

In [92]:
y.isna().sum()

0

In [93]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 4.   Model Selection

In [94]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error


# Define the XGBoost model
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)

# Handle missing values
imputer = SimpleImputer(strategy='mean')
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)

# Handle infinite values
X_train_imputed = np.nan_to_num(X_train_imputed, nan=np.nan, posinf=np.nan, neginf=np.nan)
X_test_imputed = np.nan_to_num(X_test_imputed, nan=np.nan, posinf=np.nan, neginf=np.nan)

# Scale your data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_imputed)
X_test_scaled = scaler.transform(X_test_imputed)

# Train the model
xgb_model.fit(X_train_scaled, y_train)

# Predict on the test set
y_pred = xgb_model.predict(X_test_scaled)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
print(f"Root Mean Squared Error: {rmse}")


Root Mean Squared Error: 94.44913046346618


In [107]:
# Predict on the testing set
submission_predictions = xgb_model.predict(test_combined)

In [108]:
submission_predictions

array([660.1254 , 801.3176 , 740.8749 , 744.27014, 764.041  , 669.07214,
       637.1385 , 722.61127, 738.718  , 765.9642 , 692.3342 , 758.0527 ,
       662.695  , 678.86194, 689.6446 , 853.78253, 600.9587 , 724.7368 ,
       798.71515, 748.1808 , 787.6073 , 686.6106 , 694.3372 , 673.25   ,
       726.0485 , 622.719  , 760.3655 , 648.06805, 649.19324, 590.0073 ,
       731.0358 , 686.7662 , 542.2732 , 855.5747 , 714.9281 , 656.78326,
       583.35565, 744.55493, 780.534  , 574.73   , 635.88904, 761.94214,
       730.69806, 704.7396 , 631.77356, 660.3307 , 647.4924 , 621.8718 ,
       672.40894, 635.4504 , 586.84406, 681.238  , 643.92694, 668.5289 ,
       663.3148 , 746.54785, 654.43365, 729.9126 , 630.7289 , 684.56116,
       695.297  , 691.11847, 701.2235 , 576.42487, 615.53436, 634.77386,
       617.3992 , 611.0928 , 771.0265 , 725.98395, 771.9873 , 729.98535,
       694.97394, 781.47296, 628.35657, 633.42957, 731.00665, 747.4929 ,
       574.15015, 630.835  , 611.9481 , 648.6943 , 

In [105]:
submission_df = pd.read_csv("data/submission/submission.csv")

In [111]:
submission_df.head()

,id,prediction
0,5829da64e4b051c58fcef11a,NaN
1,587b9b50e4b08f2486cd58c1,NaN
2,5789ce5de4b0022659314a39,NaN
3,58bf17bae4b0bec08aa033f9,NaN
4,58418a68e4b0bb9d6a0d716b,NaN


In [115]:
# Create a new DataFrame for predictions

submission_df['prediction'] = submission_predictions

In [116]:
submission_df.head()

,id,prediction
0,5829da64e4b051c58fcef11a,660.125427
1,587b9b50e4b08f2486cd58c1,801.317627
2,5789ce5de4b0022659314a39,740.874878
3,58bf17bae4b0bec08aa033f9,744.270142
4,58418a68e4b0bb9d6a0d716b,764.041016


In [118]:
submission_df.to_csv('data/submission/submission_final.csv', index=False)

In [119]:
final = pd.read_csv("data/submission/submission_final.csv")

In [120]:
final.shape

(1000, 2)